# Proyecto Final Big Data

Entregable esperado

Se espera que cada uno de los grupos conformados entreguen una arquitectura de datalake implementada en AWS con lo siguiente:

1. Dos orígenes de datos integrados, al menos uno de esos orígenes debe ser una tabla de una base de datos relacional, las opciones de motores de bases de datos son: MySql, Posgress, Oracle y SqlServer. (+1)
2. La integración de dichos orígenes debe ser de forma automática y usted debe escoger el método. (importante, si lo hace por pdi o un programa de integración tiene +0.5, si lo hace con código tiene +1)
3. Los datos integrados deben poderse consultar por medio de un query engine (Hive, Pig, Impala, Athena, Redshift, etc). (+1)
4. Recuerde usar las buenas prácticas enseñadas en clase (ELT, Encoding, nombres, etc) (+1)
5. El proyecto se hace en grupos, pero lo sustenta uno solo. La persona que sustenta el proyecto es de libre elección del docente. (+1)


## Introducción del Proyecto

### Descripción General

El objetivo de este proyecto es desarrollar una arquitectura de **Data Lake** utilizando **Amazon Web Services (AWS)**, donde se integren y procesen datos provenientes de diferentes fuentes. Como parte del proyecto, se utilizarán datos de la base de datos **Northwind**, un clásico dataset que representa un sistema de gestión de ventas, y se integrarán con datos almacenados en archivos CSV. El propósito es permitir consultas eficientes sobre los datos usando **Amazon Athena**.

### Tablas Utilizadas

Para este proyecto, utilizamos tres tablas principales de la base de datos Northwind (orders y products) y un archivo CSV (order_details):

#### 1. **Orders**
Esta tabla contiene información sobre los pedidos realizados por los clientes. Las columnas clave son:

- **order_id**: Identificador único del pedido.
- **customer_id**: Identificador del cliente que realizó el pedido.
- **employee_id**: Identificador del empleado que gestionó el pedido.
- **order_date**: Fecha en la que se realizó el pedido.
- **required_date**: Fecha en la que el pedido debería ser entregado.
- **shipped_date**: Fecha en la que el pedido fue enviado.
- **ship_via**: Método de envío utilizado para el pedido.
- **freight**: Costo de envío del pedido.
- **ship_name**: Nombre del destinatario del pedido.
- **ship_address**: Dirección de envío.
- **ship_city**: Ciudad de destino del pedido.
- **ship_postal_code**: Código postal del lugar de destino.
- **ship_country**: País de destino del pedido.

#### 2. **Order Details**
Esta tabla contiene información detallada de cada producto en los pedidos. Las columnas principales son:

- **order_id**: Identificador único del pedido (llave foránea de la tabla `Orders`).
- **product_id**: Identificador único del producto (llave foránea de la tabla `Products`).
- **unit_price**: Precio unitario del producto en el pedido.
- **quantity**: Cantidad del producto solicitado.
- **discount**: Descuento aplicado al producto.

Esta tabla se encuentra almacenada en un archivo CSV y se integrará con las otras tablas del proyecto.

#### 3. **Products**
La tabla `Products` contiene la información de los productos disponibles en el sistema. Sus principales columnas son:

- **product_id**: Identificador único del producto.
- **product_name**: Nombre del producto.
- **supplier_id**: Identificador del proveedor del producto.
- **category_id**: Identificador de la categoría a la que pertenece el producto.
- **quantity_per_unit**: Descripción de la cantidad de producto por unidad.
- **unit_price**: Precio unitario del producto.
- **units_in_stock**: Cantidad de unidades disponibles en inventario.
- **units_on_order**: Cantidad de unidades ordenadas que aún no han sido recibidas.
- **reorder_level**: Nivel de stock en el que se debe realizar una nueva orden.
- **discontinued**: Indica si el producto está descontinuado.



## Desarollo del Proyecto

### 1. Carga de las Librerías y Configuración del Entorno

En esta primera sección, se cargan las librerías necesarias para trabajar con **PostgreSQL**, **Pandas** para la manipulación de datos y **Boto3** para la conexión con servicios de **AWS**. Además, se utiliza la librería **dotenv** para cargar las variables de entorno desde un archivo `.env`, lo que facilita la configuración de las credenciales de acceso a la base de datos y los servicios de AWS.

In [4]:
# Importar las librerías necesarias
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine
import boto3
from botocore.exceptions import NoCredentialsError

# Cargar el archivo .env
load_dotenv()

# Obtener las variables de entorno
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST', 'localhost')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB')

# Crear el string de conexión usando SQLAlchemy
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'

# URL de conexión usando las variables de entorno
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}")

### 2. Conexión a la Base de Datos y Carga de las Tablas
En esta sección, se establece la conexión con la base de datos PostgreSQL usando SQLAlchemy y las variables de entorno cargadas. Se extraen las tablas orders y products de la base de datos y se almacenan en DataFrames de Pandas para su posterior análisis y manipulación.

In [5]:
# Conectar a la base de datos y cargar las tablas orders y products
try:
    orders_df = pd.read_sql('SELECT * FROM orders', con=engine)
    products_df = pd.read_sql('SELECT * FROM products', con=engine)
    print("Tablas cargadas exitosamente")
except Exception as e:
    print(f"Error conectando a la base de datos o cargando las tablas: {e}")

# Mostrar las primeras filas de cada tabla
print(orders_df.shape)
print(products_df.shape)

Tablas cargadas exitosamente
(830, 14)
(77, 10)


In [6]:
# Verificar dataframe orders
orders_df

,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,Mexico
826,11074,SIMOB,7,1998-05-06,1998-06-03,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,1998-05-06,1998-06-03,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Switzerland
828,11076,BONAP,4,1998-05-06,1998-06-03,None,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France


In [7]:
# Verificar dataframe products
products_df

,product_id,product_name,supplier_id,category_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,reorder_level,discontinued
0,1,Chai,8,1,10 boxes x 30 bags,18.00,39,0,10,1
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,1
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,0


### 3. Carga del Archivo CSV y Unión de Datos
En este paso, se cargan los datos del archivo CSV que contiene la tabla order_details.

In [8]:
# Cargar los datos del archivo CSV
order_details_df = pd.read_csv('data/order_details.csv', sep=';')

# Ver las primeras filas del CSV
order_details_df.head()

,order_id,product_id,unit_price,quantity,discount
0,11060,60,34.0,4,0.0
1,11060,77,13.0,10,0.0
2,11061,60,34.0,15,0.0
3,11062,53,32.8,10,0.2
4,11062,70,15.0,12,0.2


### 4. Integración de los datos usando las llaves correspondientes
Ahora, se realiza la unión de las tres tablas (orders, order_details y products) usando las llaves order_id y product_id para crear una tabla unificada con toda la información relevante:

* orders.order_id con order_details.order_id
* products.product_id con order_details.product_id

In [9]:
# Unión de las tablas
merged_df = pd.merge(order_details_df, orders_df, on='order_id', how='inner')
merged_df = pd.merge(merged_df, products_df, on='product_id', how='inner')

# Ver las primeras filas de la tabla resultante
merged_df.head()

,order_id,product_id,unit_price_x,quantity,discount,customer_id,employee_id,order_date,required_date,shipped_date,...,ship_country,product_name,supplier_id,category_id,quantity_per_unit,unit_price_y,units_in_stock,units_on_order,reorder_level,discontinued
0,11060,60,34.0,4,0.0,FRANS,2,1998-04-30,1998-05-28,1998-05-04,...,Italy,Camembert Pierrot,28,4,15 - 300 g rounds,34.0,19,0,0,0
1,11060,77,13.0,10,0.0,FRANS,2,1998-04-30,1998-05-28,1998-05-04,...,Italy,Original Frankfurter grüne Soße,12,2,12 boxes,13.0,32,0,15,0
2,11061,60,34.0,15,0.0,GREAL,4,1998-04-30,1998-06-11,None,...,USA,Camembert Pierrot,28,4,15 - 300 g rounds,34.0,19,0,0,0
3,11062,53,32.8,10,0.2,REGGC,4,1998-04-30,1998-05-28,None,...,Italy,Perth Pasties,24,6,48 pieces,32.8,0,0,0,1
4,11062,70,15.0,12,0.2,REGGC,4,1998-04-30,1998-05-28,None,...,Italy,Outback Lager,7,1,24 - 355 ml bottles,15.0,15,10,30,0


### 5. Guardado del Resultado en un CSV
Después de realizar la integración de las tablas, el resultado es almacenado en un nuevo archivo CSV llamado merged_orders.csv. Este archivo será posteriormente cargado en un bucket de S3 para su consulta en Amazon Athena.

In [10]:
# Guardar la tabla integrada en un nuevo archivo CSV
import time
timestamp = int(time.time())
csv_name = f'merged_orders_{timestamp}.csv'
merged_df.to_csv(f'data/{csv_name}', index=False)

### 6. Subida del Archivo a Amazon S3
En esta sección, se configura la conexión a Amazon S3 usando las credenciales almacenadas en las variables de entorno. Se define el bucket y la ruta donde se guardará el archivo resultante. Si el archivo no existe o las credenciales son incorrectas, se manejarán los errores correspondientes.

In [11]:
# Obtener las variables de entorno para S3
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME')

# Verificar que todas las variables se han cargado correctamente
assert AWS_ACCESS_KEY_ID is not None, "La variable AWS_ACCESS_KEY_ID no está definida en el archivo .env"
assert AWS_SECRET_ACCESS_KEY is not None, "La variable AWS_SECRET_ACCESS_KEY no está definida en el archivo .env"
assert S3_BUCKET_NAME is not None, "La variable S3_BUCKET_NAME no está definida en el archivo .env"

# Conexión a S3 usando las credenciales de las variables de entorno
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

# Definir el bucket y la ruta dentro de la carpeta 'proyecto_final'
s3_file_name = f'proyecto_final/{csv_name}'

# Cargar el archivo a S3
try:
    s3.upload_file(f'data/{csv_name}', S3_BUCKET_NAME, s3_file_name)
    print(f'Archivo {s3_file_name} subido exitosamente a {S3_BUCKET_NAME}')
except FileNotFoundError:
    print('El archivo no fue encontrado')
except NoCredentialsError:
    print('Credenciales de AWS no disponibles')

Archivo proyecto_final/merged_orders_1729652007.csv subido exitosamente a 2024-02-grupo4-acm


### 7. Creación de la Base de Datos en Amazon Athena
En este paso, se crea una base de datos en Amazon Athena llamada proyecto_final_db, donde se almacenarán las consultas sobre los datos que están en el bucket de S3. Esto permite la consulta eficiente de los datos integrados sin necesidad de cargarlos manualmente en un sistema de base de datos.

In [12]:
# Obtener las variables de entorno para Athena y S3
ATHENA_RESULT_BUCKET = os.getenv('ATHENA_RESULT_BUCKET')
S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME')

# Cliente de Athena
athena = boto3.client('athena', region_name='us-east-1')

# Crear base de datos (puedes cambiar el nombre 'proyecto_final_db' a uno adecuado)
database_name = 'proyecto_final_db'

# Query para crear la base de datos
create_database_query = f"""
CREATE DATABASE IF NOT EXISTS {database_name}
"""

# Ejecutar la consulta de creación de la base de datos
response = athena.start_query_execution(
    QueryString=create_database_query,
    ResultConfiguration={'OutputLocation': ATHENA_RESULT_BUCKET}
)

query_execution_id = response['QueryExecutionId']
status = athena.get_query_execution(QueryExecutionId=query_execution_id)
print(f"Estado de la consulta de creación de la base de datos: {status['QueryExecution']['Status']['State']}")

Estado de la consulta de creación de la base de datos: QUEUED


### 8. Creación de la Tabla en Amazon Athena
Finalmente, se crea una tabla externa en Amazon Athena apuntando a los datos almacenados en S3. Esta tabla tiene el esquema unificado de las tres tablas originales (orders, order_details, products) y permite realizar consultas utilizando SQL directamente sobre los datos en el bucket de S3.

In [13]:
# Crear tabla en Athena apuntando al archivo CSV en S3
table_name = 'ordenes_integradas'

create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
  order_id SMALLINT,
  product_id SMALLINT,
  unit_price DOUBLE,
  quantity SMALLINT,
  discount DOUBLE,
  customer_id STRING,
  employee_id SMALLINT,
  order_date STRING,
  required_date STRING,
  shipped_date STRING,
  ship_via SMALLINT,
  freight DOUBLE,
  ship_name STRING,
  ship_address STRING,
  ship_city STRING,
  ship_region STRING,
  ship_postal_code STRING,
  ship_country STRING,
  product_name STRING,
  supplier_id SMALLINT,
  category_id SMALLINT,
  quantity_per_unit STRING,
  product_unit_price DOUBLE,
  units_in_stock SMALLINT,
  units_on_order SMALLINT,
  reorder_level SMALLINT,
  discontinued INTEGER
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ','
)
LOCATION 's3://{S3_BUCKET_NAME}/proyecto_final/'
TBLPROPERTIES (
  'has_encrypted_data'='false',
  'skip.header.line.count'='1'
);
"""

# Ejecutar la consulta para crear la tabla
response = athena.start_query_execution(
    QueryString=create_table_query,
    QueryExecutionContext={'Database': database_name},
    ResultConfiguration={'OutputLocation': ATHENA_RESULT_BUCKET}
)

query_execution_id = response['QueryExecutionId']
status = athena.get_query_execution(QueryExecutionId=query_execution_id)
print(f"Estado de la consulta de creación de la tabla: {status['QueryExecution']['Status']['State']}")

Estado de la consulta de creación de la tabla: QUEUED


### 9. Creación de Código para Consultas en Amazon Athena

A continuación, procedemos a crear unas funciones que nos van a permitir realizar consultas en Amazon Athena. Estas funciones nos permitirán ejecutar consultas SQL sobre los datos almacenados en el bucket de S3 y obtener los resultados de forma programática.

In [19]:
# Función para ejecutar una consulta en Athena
def execute_athena_query(query, database_name):
    """
    Ejecuta una consulta en Athena y retorna el query_execution_id.
    """
    response = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database_name},
        ResultConfiguration={'OutputLocation': ATHENA_RESULT_BUCKET}
    )
    return response['QueryExecutionId']

# Función para verificar el estado de la consulta
def check_query_status(query_execution_id):
    """
    Verifica el estado de una consulta en Athena. 
    Espera hasta que la consulta haya finalizado (SUCCEEDED o FAILED).
    """
    status = 'RUNNING'
    while status == 'RUNNING':
        response = athena.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status == 'FAILED':
            raise Exception("La consulta falló")
        elif status == 'SUCCEEDED':
            print("La consulta fue exitosa")
        time.sleep(2)  # Esperar 2 segundos entre las verificaciones

# Función para obtener y convertir los resultados en un DataFrame de Pandas
def fetch_query_results(query_execution_id):
    """
    Obtiene los resultados de una consulta de Athena y los devuelve como un DataFrame de Pandas.
    """
    results = athena.get_query_results(QueryExecutionId=query_execution_id)
    
    # Procesar las filas y columnas
    rows = results['ResultSet']['Rows']
    columns = [col['VarCharValue'] for col in rows[0]['Data']]
    data = [[col.get('VarCharValue', '') for col in row['Data']] for row in rows[1:]]
    
    # Crear DataFrame de Pandas
    df = pd.DataFrame(data, columns=columns)
    return df

# Función para ejecutar cualquier consulta en Athena y obtener los resultados
def run_custom_query(query):
    """
    Ejecuta una consulta SQL en Athena y retorna los resultados como un DataFrame.
    
    Parámetros:
    - query (str): La consulta SQL a ejecutar.
    
    Retorna:
    - DataFrame de Pandas con los resultados de la consulta.
    """
    database_name = 'proyecto_final_db'  # Puedes cambiar el nombre de la base de datos según sea necesario
    
    # Ejecutar la consulta
    query_execution_id = execute_athena_query(query, database_name)
    
    # Verificar el estado de la consulta
    check_query_status(query_execution_id)
    
    # Obtener los resultados y mostrarlos en un DataFrame
    df = fetch_query_results(query_execution_id)
    
    # Mostrar los resultados
    print("Resultados de la consulta:")
    
    return df

### 10. Ejecutar Consultas en Amazon Athena

Ahora procedemos a usar las funciones creadas para ejecutar consultas en Amazon Athena y obtener los resultados. Realizamos algunas consultas de ejemplo para mostrar cómo se pueden obtener datos de las tablas integradas y responder preguntas sobre los pedidos y productos.

In [20]:
# Consulta para obtener las primeras 10 filas de la tabla "ordenes_integradas".
# Esta consulta selecciona todas las columnas y devuelve solo las primeras 10 filas.
# Útil para verificar el contenido de la tabla de manera rápida.

query = """
SELECT * 
FROM ordenes_integradas 
LIMIT 10;
"""

# Ejecutar la consulta
run_custom_query(query)

Resultados de la consulta:


,order_id,product_id,unit_price,quantity,discount,customer_id,employee_id,order_date,required_date,shipped_date,...,ship_country,product_name,supplier_id,category_id,quantity_per_unit,product_unit_price,units_in_stock,units_on_order,reorder_level,discontinued
0,11060,60,34.0,4,0.0,FRANS,2,1998-04-30,1998-05-28,1998-05-04,...,Italy,Camembert Pierrot,28,4,15 - 300 g rounds,34.0,19,0,0,0
1,11060,77,13.0,10,0.0,FRANS,2,1998-04-30,1998-05-28,1998-05-04,...,Italy,Original Frankfurter grüne Soße,12,2,12 boxes,13.0,32,0,15,0
2,11061,60,34.0,15,0.0,GREAL,4,1998-04-30,1998-06-11,,...,USA,Camembert Pierrot,28,4,15 - 300 g rounds,34.0,19,0,0,0
3,11062,53,32.8,10,0.2,REGGC,4,1998-04-30,1998-05-28,,...,Italy,Perth Pasties,24,6,48 pieces,32.8,0,0,0,1
4,11062,70,15.0,12,0.2,REGGC,4,1998-04-30,1998-05-28,,...,Italy,Outback Lager,7,1,24 - 355 ml bottles,15.0,15,10,30,0
5,11063,34,14.0,30,0.0,HUNGO,3,1998-04-30,1998-05-28,1998-05-06,...,Ireland,Sasquatch Ale,16,1,24 - 12 oz bottles,14.0,111,0,15,0
6,11063,40,18.4,40,0.1,HUNGO,3,1998-04-30,1998-05-28,1998-05-06,...,Ireland,Boston Crab Meat,19,8,24 - 4 oz tins,18.4,123,0,30,0
7,11063,41,9.65,30,0.1,HUNGO,3,1998-04-30,1998-05-28,1998-05-06,...,Ireland,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0
8,11064,17,39.0,77,0.1,SAVEA,1,1998-05-01,1998-05-29,1998-05-04,...,USA,Alice Mutton,7,6,20 - 1 kg tins,39.0,0,0,0,1
9,11064,41,9.65,12,0.0,SAVEA,1,1998-05-01,1998-05-29,1998-05-04,...,USA,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0


In [21]:
# Consulta para calcular el total de ventas (freight) por país.
# Agrupa los resultados por el país de envío (ship_country) y suma los costos de envío (freight) de cada país.
# Los resultados se ordenan de manera descendente según el total de ventas, mostrando los 10 países con mayores ventas.

query = """
SELECT ship_country, SUM(freight) AS total_freight
FROM ordenes_integradas
GROUP BY ship_country
ORDER BY total_freight DESC
LIMIT 10;
"""

# Ejecutar la consulta
run_custom_query(query)

Resultados de la consulta:


,ship_country,total_freight
0,USA,92331.63999999991
1,Germany,76341.50000000007
2,Austria,55462.88000000004
3,Sweden,20292.240000000013
4,UK,16981.059999999998
5,Venezuela,15744.879999999992
6,Ireland,14428.980000000003
7,05487-020,13932.879999999997
8,Canada,12643.799999999994
9,Denmark,8572.960000000003


In [22]:
# Consulta para obtener la cantidad total de productos vendidos por categoría (category_id).
# Agrupa los resultados por el ID de la categoría y suma la cantidad de productos vendidos (quantity).
# Los resultados se ordenan de manera descendente según la cantidad total vendida, mostrando las 10 categorías con mayor cantidad de productos vendidos.

query = """
SELECT category_id, SUM(quantity) AS total_quantity
FROM ordenes_integradas
GROUP BY category_id
ORDER BY total_quantity DESC
LIMIT 10;
"""

# Ejecutar la consulta
run_custom_query(query)

Resultados de la consulta:


,category_id,total_quantity
0,4,16354
1,1,15672
2,8,14136
3,3,13626
4,2,9476
5,5,7664
6,6,7412
7,7,6528
8,12,1544
9,24,856


In [23]:
# Consulta para calcular el descuento promedio aplicado por cada empleado.
# Agrupa los resultados por el ID del empleado (employee_id) y calcula el promedio de descuento (discount).
# Los resultados se ordenan de manera descendente mostrando los empleados que aplicaron mayores descuentos en promedio.

query = """
SELECT employee_id, AVG(discount) AS avg_discount
FROM ordenes_integradas
GROUP BY employee_id
ORDER BY avg_discount DESC;
"""

# Ejecutar la consulta
run_custom_query(query)

Resultados de la consulta:


,employee_id,avg_discount
0,7,0.07357954545454543
1,9,0.06822429906542059
2,5,0.06666666666666667
3,4,0.06130952380952381
4,8,0.056153846153846144
5,6,0.05446428571428569
6,3,0.04968847352024922
7,1,0.04924637681159422
8,2,0.04377593360995849


In [24]:
# Consulta para obtener todos los pedidos realizados entre dos fechas específicas.
# La consulta selecciona todas las columnas de la tabla "ordenes_integradas" y filtra los resultados
# para mostrar solo aquellos pedidos cuyo "order_date" esté entre '1998-01-01' y '1998-03-31'.

query = """
SELECT * 
FROM ordenes_integradas
WHERE order_date BETWEEN '1998-01-01' AND '1998-03-31';
"""

# Ejecutar la consulta
run_custom_query(query)

Resultados de la consulta:


,order_id,product_id,unit_price,quantity,discount,customer_id,employee_id,order_date,required_date,shipped_date,...,ship_country,product_name,supplier_id,category_id,quantity_per_unit,product_unit_price,units_in_stock,units_on_order,reorder_level,discontinued
0,10953,20,81.0,50,0.05,AROUT,9,1998-03-16,1998-03-30,1998-03-25,...,UK,Sir Rodney's Marmalade,8,3,30 gift boxes,81.0,40,0,0,0
1,10953,31,12.5,50,0.05,AROUT,9,1998-03-16,1998-03-30,1998-03-25,...,UK,Gorgonzola Telino,14,4,12 - 100 g pkgs,12.5,0,70,20,0
2,10954,16,17.45,28,0.15,LINOD,5,1998-03-17,1998-04-28,1998-03-20,...,Venezuela,Pavlova,7,3,32 - 500 g boxes,17.45,29,0,10,0
3,10954,31,12.5,25,0.15,LINOD,5,1998-03-17,1998-04-28,1998-03-20,...,Venezuela,Gorgonzola Telino,14,4,12 - 100 g pkgs,12.5,0,70,20,0
4,10954,45,9.5,30,0.0,LINOD,5,1998-03-17,1998-04-28,1998-03-20,...,Venezuela,Rogede sild,21,8,1k pkg.,9.5,5,70,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,10845,23,9.0,70,0.1,QUICK,8,1998-01-21,1998-02-04,1998-01-30,...,Germany,Tunnbröd,9,5,12 - 250 g pkgs.,9.0,61,0,25,0
900,10845,35,18.0,25,0.1,QUICK,8,1998-01-21,1998-02-04,1998-01-30,...,Germany,Steeleye Stout,16,1,24 - 12 oz bottles,18.0,20,0,15,0
901,10845,42,14.0,42,0.1,QUICK,8,1998-01-21,1998-02-04,1998-01-30,...,Germany,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.0,26,0,0,1
902,10845,58,13.25,60,0.1,QUICK,8,1998-01-21,1998-02-04,1998-01-30,...,Germany,Escargots de Bourgogne,27,8,24 pieces,13.25,62,0,20,0


In [25]:
# Consulta para calcular los ingresos generados por cada producto.
# Multiplica la cantidad de productos vendidos (quantity) por el precio unitario (unit_price)
# y agrupa los resultados por producto, mostrando los productos con mayores ingresos generados.

query = """
SELECT product_name, SUM(quantity * unit_price) AS total_revenue
FROM ordenes_integradas
GROUP BY product_name
ORDER BY total_revenue DESC
LIMIT 10;
"""

# Ejecutar la consulta
run_custom_query(query)

Resultados de la consulta:


,product_name,total_revenue
0,Côte de Blaye,239890.4
1,Brazil,229936.96000000008
2,Thüringer Rostbratwurst,150222.84000000003
3,France,147492.82
4,Raclette Courdavault,136686.0
5,Tarte au sucre,84871.40000000001
6,Camembert Pierrot,84183.99999999999
7,Manjimup Dried Apples,72991.6
8,Gnocchi di nonna Alice,72610.40000000001
9,Alice Mutton,56924.4
